In [1]:
!pip install autogluon.tabular


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempti

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile

In [3]:


# Unzip the file
with zipfile.ZipFile('/content/project_2_training.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('')

# Load the dataset
df = pd.read_csv('project_2_training.csv').sample(10000) # sample it to speed things up!

# Display the first few rows of the dataframe
df.head()


FileNotFoundError: [Errno 2] No such file or directory: '/content/project_2_training.csv.zip'

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into train and test
train_data, test_data,  = train_test_split(df, test_size=0.3, random_state=42)
print(f"X_train: {train_data.shape}")
print(f"X_test: {test_data.shape}")


In [ ]:
df['EVENT_LABEL'] = df['EVENT_LABEL'].map({'legit': 0, 'fraud': 1})
df['EVENT_LABEL'].value_counts()

In [ ]:
time_limit = 60           # for quick demonstration only, you should set this to longest time you are willing to wait (in seconds)
metric = 'roc_auc'        # specify your evaluation metric here

# Columns to exclude
ignored_columns = ['EVENT_ID','EVENT_TIMESTAMP' ]
train_data.drop(columns=ignored_columns, inplace=True)
test_data.drop(columns=ignored_columns, inplace=True)

# Initialize the TabularPredictor object.
glu_model = TabularPredictor(label='EVENT_LABEL',
                             problem_type='binary',
                             eval_metric=metric)


# Fit the model.
"""
presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
"""
glu_model = glu_model.fit(train_data,
                          time_limit=time_limit,
                          presets='medium_quality', # switch to one of the above presets.
                          )

In [ ]:
glu_model.evaluate(test_data)

In [ ]:
glu_model.leaderboard(test_data)


In [ ]:
glu_model.leaderboard(test_data, extra_metrics=['accuracy', 'balanced_accuracy', 'log_loss'])


In [ ]:
all_models = glu_model.model_names()
model_to_use = all_models[-1]
specific_model = glu_model._trainer.load_model(model_to_use)

# Objects defined below are dicts of various information (not printed here as they are quite large):
model_info = specific_model.get_info()
predictor_information = glu_model.info()

In [ ]:
predictor_information


In [ ]:
print("AutoGluon infers problem type is: ", glu_model.problem_type)
print("AutoGluon identified the following types of features:")
print(glu_model.feature_metadata)

In [ ]:
# To transform the data into AutoGluon’s internal representation, we can do the following:
test_data_transform = glu_model.transform_features(test_data)
test_data_transform.head()

In [ ]:
glu_model.feature_importance(test_data)

In [ ]:
# best model
glu_model.model_best

In [ ]:
glu_model.leaderboard(extra_info=True)

In [ ]:
all_models = glu_model.model_names()
print(f"All models: {all_models}")


In [ ]:
model_to_use = all_models[-1]
specific_model = glu_model._trainer.load_model(model_to_use)

# Objects defined below are dicts of various information (not printed here as they are quite large):
model_info = specific_model.get_info()
predictor_information = glu_model.info()

In [ ]:
model_info.keys()

In [ ]:
model_info['stacker_info']

In [ ]:
predictor_information['model_info']['WeightedEnsemble_L2']['children_info']['S1F1']['model_weights']

In [ ]:
predictor_information['model_info']['WeightedEnsemble_L2']

In [ ]:
datapoint = test_data.iloc[[0]]
#print(datapoint)
glu_model.predict_proba(datapoint)

Decision Threshold Calibration
Major metric score improvements can be achieved in binary classification for metrics such as "f1" and "balanced_accuracy" by adjusting the prediction decision threshold via calibrate_decision_threshold to a value other than the default 0.5.

Below is an example of the "f1" score achieved on the test data with and without calibrating the decision threshold:

In [ ]:
print(f'Prior to calibration (predictor.decision_threshold={glu_model.decision_threshold}):')
scores = glu_model.evaluate(test_data)

calibrated_decision_threshold = glu_model.calibrate_decision_threshold()
glu_model.set_decision_threshold(calibrated_decision_threshold)

print(f'After calibration (predictor.decision_threshold={glu_model.decision_threshold}):')
scores_calibrated = glu_model.evaluate(test_data)

In [ ]:
for metric_name in scores:
    metric_score = scores[metric_name]
    metric_score_calibrated = scores_calibrated[metric_name]
    decision_threshold = glu_model.decision_threshold
    print(f'decision_threshold={decision_threshold:.3f}\t| metric="{metric_name}"'
          f'\n\ttest_score uncalibrated: {metric_score:.4f}'
          f'\n\ttest_score   calibrated: {metric_score_calibrated:.4f}'
          f'\n\ttest_score        delta: {metric_score_calibrated-metric_score:.4f}')

In [ ]:
glu_model.features()

In [ ]:
glu_model.predict_proba(test_data)[1]

In [ ]:
from sklearn.metrics import roc_curve
import numpy as np
y_test = test_data['EVENT_LABEL']
# Predict probabilities for the positive class
y_scores = glu_model.predict_proba(test_data)[1]
fpr, tpr, thresholds = roc_curve(test_data['EVENT_LABEL'], y_scores)
# Define target FPR values
target_fpr = np.arange(0.01, 0.11, 0.01)  # From 1% to 10%

# Interpolate to find TPR and threshold for target FPRs
interp_tpr = np.interp(target_fpr, fpr, tpr)
interp_thresholds = np.interp(target_fpr, fpr, thresholds)

# Print the results
for i in range(len(target_fpr)):
    print(f"Target FPR: {target_fpr[i]:.2f}, Expected TPR: {interp_tpr[i]:.4f}, Threshold: {interp_thresholds[i]:.4f}")


In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
ks_statistic = max(tpr - fpr)
ks_index = np.argmax(tpr - fpr)  # Index of the maximum difference
ks_threshold = thresholds[ks_index]  # Threshold at the maximum difference

print(f"KS Statistic: {ks_statistic:.4f}, at threshold: {ks_threshold:.4f}")

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

# Convert probabilities to binary predictions based on the KS threshold
y_pred_ks_threshold = (y_scores >= ks_threshold).astype(int)
# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred_ks_threshold)
tn, fp, fn, tp = cm.ravel()

# Calculate metrics
recall = tp / (tp + fn)  # Also known as sensitivity or TPR
precision = tp / (tp + fp)
fpr = fp / (fp + tn)

print(f"Recall (TPR): {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"False Positive Rate (FPR): {fpr:.4f}")


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

# Assuming y_test are your true labels and y_scores are the predicted probabilities for the positive class
roc_auc = roc_auc_score(y_test, y_scores)
print(roc_auc)
fpr, tpr, thresholds = roc_curve(y_test, y_scores)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)  # Assuming you've calculated roc_auc elsewhere
plt.plot([0, 1], [0, 1], 'k--')  # Random chance line
plt.scatter(fpr[ks_index], tpr[ks_index], color='red')  # KS statistic point

# Annotate the KS statistic on the plot
plt.annotate(f'KS={ks_statistic:.4f}', xy=(fpr[ks_index], tpr[ks_index]), xytext=(fpr[ks_index]+0.1, tpr[ks_index]-0.1),
             arrowprops=dict(facecolor='red', shrink=0.05))

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()
